### PDF Reader Code using PDFMiner

* Data Cleaning by removing stopwords and stemming words.
* Storing this text data into txt_files

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

ps = PorterStemmer() 

try:
    os.mkdir('TextFilesAllPapers')
except:
    pass

def data_cleaner(text, path):
  
           
        # Gets filename from the path
        file = os.path.basename(path.split('.')[0])
        
        #print("Started converting ",text)
        
        text = text.replace('\n', ' ')
        text = re.sub('[^a-zA-Z0-9 \n\.]', ' ', text).lower()
        #stopwords removed
        text = ' '.join(word for word in text.split() if word not in stop_words)

        token_words=word_tokenize(text)
        new=[]
        for word in token_words:
            new.append(ps.stem(word))
            new.append(" ")
        new_text= "".join(new)

        filename= file+".txt"
        file = open("./TextFilesAllPapers/"+filename, "w+") 
        file.write(new_text) 
        file.close()

In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import os 
import re
import xlsxwriter
from nltk.tokenize import sent_tokenize

def convert_pdf_to_txt(path, pages=None):
  
  #checks existence of pages and gets the number of pages
    if not pages:
        pagenums = set()
        
    else:
        pagenums = set(pages)
        
   # sets parameters for the miner    
    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)
  
  # iterates through the pages of pdf and puts it in a variable
    infile = open(path, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close()
    
    # Creates a txt file of the stemmed text
    data_cleaner(text, path)
    
    return text

### Extracting the KPI into excel files

In [2]:
#This is a list of words that will be searched only if other KPI's are found as they are ambiguous

special_words = ['share', 'margin', 'profitability', 'valuation', 'acquisitions']

# distinctwords is the list of investment related KPI words
with open(r"distinctwords.txt", "r") as cmWords:
    content = cmWords.readlines()

#appends the words from the txt files in a list    
newCon =[]
for word in content:
    newCon.append((word.lower().replace('\n','')))

# creates a new excel file if not exists for storing KPI's
workbook = xlsxwriter.Workbook('KPI_extraction_file.xlsx')

#keeps a count of the number of documents that are investment related
count = 0


filesN=[]

# finds all pdf documents in the folder
for top, dirs, files in os.walk(r'ab1All papers'):
    for filename in files:
        if filename.endswith('.pdf'):
            kpi_list ={}
            try:
                print("started "+filename)
                abspath = os.path.join(top, filename)
                text = convert_pdf_to_txt(abspath)
                text = sent_tokenize(text, "english")
                #text = (text.lower()).split('.')
                count = 0
                
                # searches for KPI related words in every sentence in the pdf
                for sent in text:
                    for everyCon in newCon:
                        mila = re.search(" "+everyCon, sent)
                        # mila = sent.find(everyCon)
                        
                        #adds the KPI in a dictionary along with the sentence if KPI not already in it
                        if(mila):
                            if everyCon not in kpi_list.keys():
                                kpi_list[everyCon] = [sent]
                            else:
                                if sent in kpi_list.values():
                                    pass
                                else:
                                    kpi_list[everyCon].append(sent)
                                
                
                temp_list = kpi_list.keys()
                temp_list2 = []
                for word in temp_list:
                  
                  if word in special_words:
                    pass
                  else:
                    temp_list2.append(word)
                    
                p = len(temp_list2)
                
                if(p > 0):
                    count = count +1
                    filesN.append(filename)
                    worksheet = workbook.add_worksheet(filename)
                    row = 0
                    col = 0
                    for key in kpi_list.keys():
                        row += 1
                        worksheet.write(row, col, key)
                        for item in kpi_list[key]:
                            worksheet.write(row, col + 1, item)
                            row += 1
                else:
                    continue

            except:
                print("IN Except "+filename)
                continue
        else:
            print("In else " + filename)
            continue
workbook.close()
print(count)

started a098ijqs.pdf
started a09jhr.pdf
started a11eee.pdf
started a12dissertation.pdf
started a94amj.pdf
started aal11tlq.pdf
started abc01o.pdf
started abk01lodj.pdf
started abs04crr.pdf
started abto06jap.pdf
started abw11jpa.pdf
started ach85amj.pdf
started ad00jom.pdf
started adg08hrm.pdf
started af80ar.pdf
started afglmm10hrm.pdf
started ah98aos.pdf
started ahnl08lodj.pdf
started alj09hrm.pdf
started aly10jom.pdf
started am79amj.pdf
started amh90ilr.pdf
started amm15ijpe.pdf
started ap81asq.pdf
started ar07lrp.pdf
started ar78amj.pdf
started as03jom.pdf
started asj03jmtp.pdf
started atc05ijhrm.pdf
started awl08jabs.pdf
started awzh12hp.pdf
started ay14ijbs.pdf
started azw06meq.pdf
started azw06meqij.pdf
started b00ema.pdf
started b01joop.pdf
started b04ilr.pdf
started b76fm.pdf
started b89ar.pdf
started b92ilrr.pdf
started b92pb.pdf
started b93joop.pdf
started b94dissertation.pdf
started b94jes.pdf
started b95jams.pdf
started b99dissertation.pdf
started b99ilr.pdf
started ba05joop

started lb98gjhr.pdf
started lchp09ijpr.pdf
started lcy12ijebm.pdf
started lh05item.pdf
started lh13jbim.pdf
started lhwh15ijlm.pdf
started lj03emj.pdf
started lk09amj.pdf
started lk11ijhm.pdf
started lkc12imds.pdf
started lkhl10ijpr.pdf
started lm96os.pdf
started lmssh02ms.pdf
started lnaa08job.pdf
started lpaf13jcp.pdf
started lrcn13md.pdf
started ls00nvsq.pdf
started ls13jpssm.pdf
started ls80pc.pdf
started lss00ijsm.pdf
started lt87jb.pdf
started ltg13jcp.pdf
started ltj10ijoa.pdf
started lw12o.pdf
started lw98hrdq.pdf
started lwl14aijpe.pdf
started lwl14bijpe.pdf
started lwl14ijpe.pdf
started lwlm14amj.pdf
started lww13ijpe.pdf
started lyt10ijopm.pdf
IN Except lyt10ijopm.pdf
started lzl11ijcm.pdf
started m00dissertation.pdf
started m06aaaj.pdf
started m06thesis.pdf
started m08amp.pdf
started m10bse.pdf
started m91dissertation.pdf
started m94dissertation.pdf
started m95ilr.pdf
started m96jom.pdf
started m97amp.pdf
started mao10jbe.pdf
started mb05ejm.pdf
started mcf03jocm.pdf
start